In [39]:
from google.cloud import bigquery
import folium

In [40]:
# Initialize BigQuery client
client = bigquery.Client()

# Bring in all data:
# ------------------

# Create function to grab each table and return a df
def get_data(table_name):
    query = f"select * from `bigquery-public-data.thelook_ecommerce.{table_name}`"
    return client.query(query).to_dataframe()

# Assign each df to a variable
table_names = ['distribution_centers', 'users', 'orders', 'order_items', 'inventory_items']
for i in range(len(table_names)):
    globals()[table_names[i]] = get_data(table_names[i])

In [41]:
# Find out if two items in an order were shipped out separately
import duckdb

query_distinct_shipments = """
select
    order_id
    , count(distinct shipped_at) as distinct_ship_times
from order_items
group by 1
having count(distinct shipped_at) > 1
order by 2 desc
"""

result = duckdb.query(query_distinct_shipments).df()
result # Answer: No, all items in an order were shipped out at the same time

,order_id,distinct_ship_times


In [42]:
# Find out how many items are in each order and their counts
query_items_per_order = """
select
    order_id
    , count(id) as num_items
from order_items
group by 1
order by 2 desc
"""
duckdb.query(query_items_per_order).df() # Number of items per order ranges from 1 to 4

,order_id,num_items
0,25437,4
1,36575,4
2,68082,4
3,90181,4
4,48513,4
...,...,...
124535,44135,1
124536,85546,1
124537,17604,1
124538,104113,1


In [43]:
# Preview order_id in order_items table
query_order_id_items = """
select *
from order_items
where order_id = 47598
"""
duckdb.query(query_order_id_items).df() # Extra validation that all the items in order 47598 were shipped at the same time

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,68896,47598,38234,25331,186258,Processing,2024-03-30 12:05:39+00:00,NaT,NaT,NaT,8.99


In [44]:
# Create dataset by joining tables

order_details = (
    order_items
    .merge(inventory_items, how='left', left_on='inventory_item_id', right_on='id', suffixes=('_order_items', '_inventory'))
    .merge(distribution_centers, how='left', left_on='product_distribution_center_id', right_on='id')
    .merge(users, how='left', left_on='user_id', right_on='id', suffixes=('', '_user'))
)

order_details.rename(columns={'id': 'id_dist_center', 'name': 'name_dist_center', 'latitude': 'latitude_dist_center', 'longitude': 'longitude_dist_center'}, inplace=True)
print(order_details.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 180466 entries, 0 to 180465
Data columns (total 44 columns):
 #   Column                          Non-Null Count   Dtype              
---  ------                          --------------   -----              
 0   id_order_items                  180466 non-null  Int64              
 1   order_id                        180466 non-null  Int64              
 2   user_id                         180466 non-null  Int64              
 3   product_id_order_items          180466 non-null  Int64              
 4   inventory_item_id               180466 non-null  Int64              
 5   status                          180466 non-null  object             
 6   created_at_order_items          180466 non-null  datetime64[ns, UTC]
 7   shipped_at                      117625 non-null  datetime64[ns, UTC]
 8   delivered_at                    63446 non-null   datetime64[ns, UTC]
 9   returned_at                     17810 non-null   datetime64[ns, UTC]
 

In [45]:
# Filter for only shipped orders
shipped_orders = order_details[order_details['status'] == 'Shipped']

# Select only relevant columns
relevant_data = shipped_orders[[
    'distribution_center_geom',
    'latitude_dist_center',
    'longitude_dist_center',
    'user_geom',
    'latitude_user',
    'longitude_user',
    'first_name', 'last_name', 
    'city', 'state',
    'name_dist_center',
    'order_id'
]]

relevant_data.reset_index(inplace=True)
relevant_data

,index,distribution_center_geom,latitude_dist_center,longitude_dist_center,user_geom,latitude_user,longitude_user,first_name,last_name,city,state,name_dist_center,order_id
0,2,POINT(-89.9711 35.1174),35.1174,-89.9711,POINT(2.898221046 42.70234107),42.702341,2.898221,Debra,Wiley,Perpignan,Occitanie,Memphis TN,7309
1,3,POINT(-89.9711 35.1174),35.1174,-89.9711,POINT(-122.9912099 45.32309358),45.323094,-122.991210,Michelle,Perez,Newberg,Oregon,Memphis TN,86841
2,4,POINT(-89.9711 35.1174),35.1174,-89.9711,POINT(-4.80601686 39.95638658),39.956387,-4.806017,Caitlin,Foster,Talavera de la Reina,Castilla-La Mancha,Memphis TN,95584
3,15,POINT(-88.0431 30.6944),30.6944,-88.0431,POINT(-59.53483513 -4.957320329),-4.957320,-59.534835,Kenneth,Sweeney,null,Amazonas,Mobile AL,61063
4,16,POINT(-88.0431 30.6944),30.6944,-88.0431,POINT(-47.9225723 -15.82675402),-15.826754,-47.922572,Martin,Perez,Brasília,Distrito Federal,Mobile AL,85780
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54174,180461,POINT(-90.0667 29.95),29.9500,-90.0667,POINT(2.360737029 48.87612795),48.876128,2.360737,Joseph,Thompson,Paris,Île-de-France,New Orleans LA,22851
54175,180462,POINT(-90.0667 29.95),29.9500,-90.0667,POINT(-40.04733317 -4.343551853),-4.343552,-40.047333,Marcus,Hardy,Santa Quitéria,Ceará,New Orleans LA,26002
54176,180463,POINT(-90.0667 29.95),29.9500,-90.0667,POINT(-117.2639164 33.62108805),33.621088,-117.263916,Travis,Morris,Wildomar,California,New Orleans LA,27662
54177,180464,POINT(-90.0667 29.95),29.9500,-90.0667,POINT(-83.9534186 43.60063776),43.600638,-83.953419,Taylor,Grant,Bangor Charter Township,Michigan,New Orleans LA,60679


In [46]:
# Create map

## Initialize map
map_center = [relevant_data['latitude_dist_center'].mean(), relevant_data['longitude_dist_center'].mean()]
map = folium.Map(location=map_center, zoom_start=4)

## Add distribution centers and user locations
for _, row in relevant_data.head(10).iterrows():
    folium.Marker(
        location=[row['latitude_dist_center'], row['longitude_dist_center']],
        popup=f"Distribution Center: {row['name_dist_center']}",
        icon=folium.Icon(color='blue', icon='industry', prefix='fa')
    ).add_to(map)

    folium.Marker(
        location=[row['latitude_user'], row['longitude_user']],
        popup=f"User: {row['first_name']} {row['last_name']}",
        icon=folium.Icon(color='green', icon='user', prefix='fa')
    ).add_to(map)

    folium.PolyLine(
        locations=[(row['latitude_dist_center'], row['longitude_dist_center']),
                   (row['latitude_user'], row['longitude_user'])],
        popup=f"User in: {row['city']}, {row['state']}\n\n Shipped from: {row['name_dist_center']}",
        color="grey",
        weight=2,
        opacity=0.7
    ).add_to(map)

map